In [2]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ, QuantumRegister
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

import qiskit as qk
import numpy as np
import matplotlib.pyplot as plt
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
import qiskit.quantum_info as qi
import qiskit.circuit.library as ql

import random
import numpy as np
from qiskit.visualization import plot_histogram
from qiskit.providers.aer.noise import NoiseModel
import qiskit.providers.aer.noise as noise
import json

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

## Számelméleti segédfüggvények

In [3]:
def cont_frac(p, q, a=[]):
    b = list(np.copy(a))
    b.append(p//q)
    if p%q == 0:
        return b
    return cont_frac(q, p-(p//q)*q, b)

def convergents(a):
    if len(a) == 1:
        return a[0], 1
    p = [a[0], 1+a[0]*a[1]]
    q = [1, a[1]]
    for i in range(2, len(a)):
        p.append(a[i]*p[i-1]+p[i-2])
        q.append(a[i]*q[i-1]+q[i-2])
    return p, q

def order_classic(a, N): 
    if gcd(a, N)[1] != 1:
        raise ValueError("a and N have to be coprime")
    s = 1
    i = 0
    while i == 0 or s != 1:
        s = (a*s) % N
        i += 1
    return i

In [4]:
def int_nth_root(A, n): #calculate floor of nth root of A
    if A<0:
        if n%2 == 0:
            raise ValueError
        return -int_nth_root(-A,n)
    if A==0:
        return 0
    n1 = n-1
    if A.bit_length() < 1024: # float(n) safe from overflow
        xk = int( round( pow(A,1.0/n) ) )
        xk = ( n1*xk + A//pow(xk,n1) )//n # Ensure xk >= floor(nthroot(A)).
    else:
        xk = 1 << -(-A.bit_length()//n) # 1 << sum(divmod(A.bit_length(),n))
                                        # power of 2 closer but greater than the nth root of A
    while True:
        d = A // pow(xk,n1)
        if xk <= d:
            return xk
        xk = (n1*xk + d)//n

In [5]:
def two_pow(A, k): # computes A**(2**k), A: L bit int
    result = np.copy(A)
    for i in range(k): # k multiplying operations
        result = result**2 
    return result  # if k < L: max O(L^3) operations

def int_pow(A, n): # computes A**n in polynomal time, A: L bit int
    n_bin = list(bin(n))
    index = n_bin.index('b')
    n_bin = n_bin[index+1:]
    l = len(n_bin) # l < log(L)
    result = 1
    for i in range(l):
        bit = int(n_bin[l-i-1])
        if bit == 1:
            result = result*two_pow(A, i)
    return result # if n < A: max O(L^4) operations

In [6]:
def two_pow_mod(A, k, N): #computes A**(2**k) mod N, A: L bit int
    result = np.copy(A)
    for i in range(k): # k multiplying operations
        result = result**2 % N
    return result  # if  k < L: max O(L^3)

def int_pow_mod(A, n, N): #computes A**n mod N in polynomal time, A: L bit integer
    n_bin = list(bin(n))
    index = n_bin.index('b')
    n_bin = n_bin[index+1:]
    l = len(n_bin) # l < log(L)
    result = 1
    for i in range(l):
        bit = int(n_bin[l-i-1])
        if bit == 1:
            result = result*two_pow_mod(A, i, N) % N
    return result # if n < A:max O(L^4) operations

In [7]:
# determines if n is probably-prime or composite using miller rabin test. Tests k times
def is_prime(n, k=20): # miller-rabin test
    if n == 2:
        return True
    for _ in range(k):
        witness = 1
        a = random.randint(2, n-1)
        if gcd(a, n)[1] != 1:
            return False
        d = n-1
        r = 0
        while d % 2 == 0:
            d = d//2
            r += 1
        s = int_pow_mod(a, d, n)
        if s == 1:
            witness = 0
        for i in range(r):
            if s == n-1:
                witness = 0
            s = s**2 % n
        if witness == 1:
            return False
    return True

def find_int_pow(N):
    n = N.bit_length()
    for i in range(2, n):
        a = int_nth_root(N, i)
        if int_pow(a, i) == N:
            return a, i
    return N, 1

## Gépek

In [8]:
simulator = qk.BasicAer.get_backend('qasm_simulator')
real1 = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits > 3,
                                            operational=True, simulator=False))
real2 = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits > 3,
                                            operational=True, simulator=False))

## A kvantumáramkörhöz használt kapuk összeállítása

In [9]:
def qft(n): # Quantum Fourier transform
    qc = QuantumCircuit(n)
    for i in range(int(n/2)):
        qc.swap(i, n-i-1)
    for i in range(n):
        qc.h(i)
        for j in range(i+1, n):
            qc.cp(2*np.pi/2**(j-i+1), j, i)
    circ = qc.to_gate()
    circ.name = 'QFT{}'.format(n)
    return circ

In [10]:
def F_add(n, a): # addition in fourier basis
    qc = QuantumCircuit(n)
    a_bin = []
    copy = a%(2**n)
    for i in range(n):
        a_bin.append(copy//2**(n-1-i))
        
        copy = copy % 2**(n-1-i)
    #qc.append(qft(n), range(n))
    rot = np.zeros(n)
    for i in range(n):
        if a_bin[n-i-1] == 1:
            for j in range(n-i):
                rot[n-j-i-1] += 2*np.pi/2**(j+1)
    for i in range(n):
        qc.p(rot[i], i)
    #qc.append(qft(n).inverse(), range(n))
    gate = qc.to_gate()
    gate.name = '{}F_add{}'.format(n, a)
    return gate

In [11]:
# 0, 1: control | range(2, n+2): addition | n+2, n+3: auxiliary
# adds in Fourier basis
def c2_mod_add(a, N):
    n = N.bit_length()
    qc = QuantumCircuit(n+4)
    qc.append(F_add(n+1, a).control(2), range(n+3))
    qc.append(F_add(n+1, N).inverse(), range(2, n+3))
    qc.append(qft(n+1).inverse(), range(2, n+3))
    qc.cx(n+2, n+3)
    qc.append(qft(n+1), range(2, n+3))
    qc.append(F_add(n+1, N).control(), [n+3]+list(range(2, n+3)))
    qc.append(F_add(n+1, a).inverse().control(2), range(n+3))
    qc.append(qft(n+1).inverse(), range(2, n+3))
    qc.x(n+2)
    qc.cx(n+2, n+3)
    qc.x(n+2)
    qc.append(qft(n+1), range(2, n+3))
    qc.append(F_add(n+1, a).control(2), range(n+3))
    gate = qc.to_gate()
    gate.name = 'cadd{}mod{}'.format(a,N)
    return gate

In [12]:
# 0: control | range(1, n+1): number to be multiplied | range(n+1, 2n+3): c_mod_add -> result
def c_mod_mult(a, N): # multiplies by "a" modulo "N"
    n = N.bit_length()
    qc = QuantumCircuit(2*n+3)
    qc.append(qft(n+1), range(n+1, 2*n+2))
    for i in range(n):
        mod = a*(2**i) % N
        qc.append(c2_mod_add(mod, N), [0,i+1]+list(range(n+1, 2*n+3)))
    qc.append(qft(n+1).inverse(), range(n+1, 2*n+2))
    gate = qc.to_gate()
    gate.name = 'cmult{}mod{}'.format(a, N)
    return gate

In [13]:
def gcd(m, n): # euclid's algorithm
    a = max(m,n)
    b = min(m,n)
    if a%b == 0:
        return a, b
    c = a % b
    return gcd(b, c) # c is the greatest common devisor

def mod_inv(a, N): # a and N have to be coprime
    if gcd(a, N)[1] != 1:
        raise ValueError("a and N have to be coprime")
    a = a%N
    k = [0, 1]
    m = [N, a]
    while m[1] != 1:
        tmp = m[0]
        p = tmp//m[1]
        m[0] = m[1]
        m[1] = tmp - p*m[1]
        tmp1 = k[0]
        k[0] = k[1]
        k[1] = tmp1 - p*k[1]
    return k[1]%N

In [14]:
def swap():
    qc = QuantumCircuit(2)
    qc.swap(0, 1)
    gate = qc.to_gate()
    gate.name = 'swap'
    return gate

# 0: control | range(1, n+1): number to be multiplied | range(n+1, 2n+3): auxiliary       
def c_U(a, N): # multiplies with 'a' mod N, 'a' and N have to be coprime
    n = N.bit_length()
    qc = QuantumCircuit(2*n+3)
    qc.append(c_mod_mult(a, N), range(2*n+3))
    for i in range(n):
        qc.append(swap().control(), [0, i+1, n+i+1])
    qc.append(c_mod_mult(mod_inv(a, N), N).inverse(), range(2*n+3))
    gate = qc.to_gate()
    gate.name = 'cU_{},{}'.format(N, a)
    return gate

In [15]:
# performs phase estimation on U gate
def Shor_phase_est(a, N, cbits=3): # cbits: 2*n+bits control qubitet használ
    n = N.bit_length()
    qc = QuantumCircuit(4*n+2+cbits)
    qc.x([2*n+cbits])
    qc.h(range(2*n+cbits))
    for i in range(2*n+cbits):
        mod = two_pow_mod(a, i, N)
        qc.append(c_U(mod, N), [2*n+cbits-1-i]+list(range(2*n+cbits, 4*n+2+cbits)))
    for i in range(n+cbits//2):
        qc.swap(i, 2*n+cbits-1-i)
    qc.append(qft(2*n+cbits).inverse(), range(2*n+cbits))
    gate = qc.to_gate()
    gate.name = 'phase_est'
    return gate

## Perióduskereső algoritmus és Shor-algoritmus

In [34]:

def order(a, N, device = Aer.get_backend('qasm_simulator'), cbits=3, basis_gates=None, noise_model=None): # order of a mod N 
    n = N.bit_length()
    qc = QuantumCircuit(4*n+5,2*n+cbits)
    qc.append(Shor_phase_est(a, N, cbits=cbits), range(4*n+2+cbits))
    qc.measure(range(2*n+cbits), range(2*n+cbits))
    m = 0
    num = 1
    i = 0
    while (m != 1 or num == 0) and i < 50:
        job = execute(qc, device, shots=1, basis_gates=basis_gates, noise_model=noise_model)
        counts = job.result().get_counts()
        estimate = 0
        bits = list(list(counts)[0])
        for j in range(2*n+cbits):
            estimate += int(bits[j])*2**(2*n+cbits-1-j)
        p, q = convergents(cont_frac(estimate, 2**(2*n+cbits)))
        q = np.array(q, dtype = int)
        p = np.array(p, dtype = int)
        r = max(q[q <= N])
        num = max(p[q <= N])
        m = int_pow_mod(a, r, N)
        
        
        # we have to make a correction if we found an even multiple of the period due to some error
        while int_pow_mod(a, r//2, N) == 1 and r//2 != 0:
            m = int_pow_mod(a, r//2, N)
            r = r//2
        i += 1
        print('r:',r)
        print('i:', i)
        if i == 50:
            raise RuntimeError
        
    return r, i

In [35]:
def Shor(N, cbits=3): # Shor's algorithm, returns list with prime factors
    factors = [] # we collect the factors here
    
    if is_prime(N, 20) == True: # cheking if N is prime
        return [N]
    
    if N%2 == 0: # cheking if N is even
        factors = [2] + Shor(N//2)
        return factors
    
    base, power = find_int_pow(N)
    if power != 1: # cheking if N is a power of a prime
        factors = power * Shor(base)
        factors.sort()
        return factors
    
    r = 1
    while r % 2 != 0 or int_pow_mod(a, r//2, N) == N-1:
        a = random.randint(2, N-2)
        print('a:',a)
        if gcd(a, N)[1] != 1:  #cheking if N and a are coprime
            devisor = gcd(a, N)[1]
            factors = Shor(devisor) + Shor(N//devisor)
            factors.sort()
            return factors
        
        r, i = order(a, N, cbits=cbits) # returns order of 'a' mod N (quantum part)
        h = int_pow_mod(a, r//2, N)
        dev1 = gcd(N, h-1)[1]
        dev2 = gcd(N, h+1)[1]
        if (N % dev2 == 0) and (dev2 != 1) and (N != dev2):
            factors = Shor(dev2) + Shor(N//dev2)
            factors.sort()
            return factors
        elif (N % dev1) == 0 and (dev1 != 1) and (N != dev1):
            factors = Shor(dev1) + Shor(N//dev1)
            factors.sort()
            return factors

In [36]:
Shor(21, cbits=0)

a: 16
r: 1
i: 1
r: 3
i: 2


[3, 7]